# 2. Few-Shot Prompting - Aprendizaje con Ejemplos

## Objetivos de Aprendizaje
- Comprender los principios del few-shot prompting
- Seleccionar y estructurar ejemplos efectivos
- Implementar patrones de entrada-salida consistentes
- Optimizar el número y calidad de ejemplos

## ¿Qué es Few-Shot Prompting?

Few-shot prompting es una técnica donde proporcionamos **ejemplos específicos** de la tarea que queremos que el modelo realice. El modelo aprende el patrón de estos ejemplos y lo aplica a nuevas entradas.

### Estructura Básica:
```
Instrucción (opcional)
Ejemplo 1: Input → Output
Ejemplo 2: Input → Output
Ejemplo 3: Input → Output
Nueva entrada: Input → ?
```

### Ventajas:
- **Consistencia**: Resultados más predecibles
- **Formato controlado**: Salidas estructuradas
- **Menos ambigüedad**: Patrones claros a seguir
- **Tareas específicas**: Excelente para casos especializados

### Consideraciones:
- **Más tokens**: Consume más espacio de contexto
- **Selección de ejemplos**: Crucial para el éxito
- **Sesgos**: Los ejemplos pueden introducir sesgos
- **Overfitting**: Puede ser demasiado específico

In [2]:
# Configuración inicial
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
import os
import json

# Configurar el modelo
llm = ChatOpenAI(
    base_url=os.getenv("OPENAI_BASE_URL"),
    api_key=os.getenv("GITHUB_TOKEN"),
    model="gpt-4o",
    temperature=0.3  # Menor temperatura para más consistencia
)

print("✓ Modelo configurado para few-shot prompting")
print("✓ Temperature reducida para mayor consistencia")

✓ Modelo configurado para few-shot prompting
✓ Temperature reducida para mayor consistencia


## Comparación: Zero-Shot vs Few-Shot

Veamos la diferencia en resultados entre ambos enfoques.

In [3]:
# Comparación directa entre enfoques
def comparar_zero_vs_few_shot():
    print("=== COMPARACIÓN: ZERO-SHOT vs FEW-SHOT ===")
    
    # Tarea: Clasificar sentimientos de reviews de productos
    nueva_review = "El producto llegó rápido pero la calidad no es lo que esperaba por el precio."
    
    # Enfoque Zero-Shot
    prompt_zero_shot = f"""Clasifica el sentimiento de esta review como Positivo, Negativo o Neutral:
    
Review: "{nueva_review}"
Sentimiento:"""
    
    # Enfoque Few-Shot
    prompt_few_shot = f"""Clasifica el sentimiento de cada review como Positivo, Negativo o Neutral:
    
Review: "Excelente producto, superó mis expectativas. Muy recomendado."
Sentimiento: Positivo
    
Review: "Llegó defectuoso y el servicio al cliente fue terrible."
Sentimiento: Negativo
    
Review: "Está bien, cumple su función pero nada especial."
Sentimiento: Neutral
    
Review: "Buena relación calidad-precio, aunque podría mejorar el diseño."
Sentimiento: Positivo
    
Review: "{nueva_review}"
Sentimiento:"""
    
    # Probar ambos enfoques
    print("\n1. ZERO-SHOT:")
    print("-" * 15)
    try:
        response_zero = llm.invoke([HumanMessage(content=prompt_zero_shot)])
        print(f"Resultado: {response_zero.content.strip()}")
    except Exception as e:
        print(f"Error: {e}")
    
    print("\n2. FEW-SHOT:")
    print("-" * 15)
    try:
        response_few = llm.invoke([HumanMessage(content=prompt_few_shot)])
        print(f"Resultado: {response_few.content.strip()}")
    except Exception as e:
        print(f"Error: {e}")
    
    print("\n=== ANÁLISIS ===")
    print("• Zero-shot: Puede ser menos consistente en formato")
    print("• Few-shot: Más probable que siga el patrón exacto")
    print("• Few-shot: Mejor para tareas con formato específico")
    
    # Análisis de tokens
    tokens_zero = len(prompt_zero_shot.split())
    tokens_few = len(prompt_few_shot.split())
    print(f"\n• Tokens zero-shot: ~{tokens_zero}")
    print(f"• Tokens few-shot: ~{tokens_few} ({tokens_few/tokens_zero:.1f}x más)")

# Ejecutar comparación
comparar_zero_vs_few_shot()

=== COMPARACIÓN: ZERO-SHOT vs FEW-SHOT ===

1. ZERO-SHOT:
---------------
Resultado: Negativo

2. FEW-SHOT:
---------------
Resultado: Neutral

=== ANÁLISIS ===
• Zero-shot: Puede ser menos consistente en formato
• Few-shot: Más probable que siga el patrón exacto
• Few-shot: Mejor para tareas con formato específico

• Tokens zero-shot: ~28
• Tokens few-shot: ~72 (2.6x más)


## Selección Estratégica de Ejemplos

La calidad y selección de ejemplos es crucial para el éxito del few-shot prompting.

In [4]:
# Técnica 1: Ejemplos Representativos
def ejemplos_representativos():
    print("=== SELECCIÓN DE EJEMPLOS REPRESENTATIVOS ===")
    
    # Tarea: Extraer información de contacto de emails
    nuevo_email = """Hola, soy María García, gerente de ventas en TechCorp. 
    Mi teléfono es +34 678 901 234 y mi email corporativo es m.garcia@techcorp.es. 
    Nos gustaría agendar una reunión para discutir nuestra propuesta."""
    
    # Buenos ejemplos: diversos y representativos
    prompt_buenos_ejemplos = f"""Extrae información de contacto de cada email en formato JSON:
    
Email: "Saludos, soy Dr. Pedro López del Hospital Central. Pueden contactarme al 915-555-0123 o p.lopez@hospital.com para cualquier consulta médica."
JSON: {{"nombre": "Dr. Pedro López", "empresa": "Hospital Central", "telefono": "915-555-0123", "email": "p.lopez@hospital.com", "cargo": "Doctor"}}
    
Email: "Ana Ruiz, desarrolladora senior en StartupXYZ. Mi número directo es 661-234-567 y mi correo personal es ana.ruiz.dev@gmail.com"
JSON: {{"nombre": "Ana Ruiz", "empresa": "StartupXYZ", "telefono": "661-234-567", "email": "ana.ruiz.dev@gmail.com", "cargo": "Desarrolladora Senior"}}
    
Email: "Contacto comercial: Luis Martín, sin empresa específica. Tel: +1-555-0199, email: luis.martin.comercial@outlook.com"
JSON: {{"nombre": "Luis Martín", "empresa": null, "telefono": "+1-555-0199", "email": "luis.martin.comercial@outlook.com", "cargo": "Comercial"}}
    
Email: "{nuevo_email}"
JSON:"""
    
    print("USANDO EJEMPLOS REPRESENTATIVOS:")
    try:
        response = llm.invoke([HumanMessage(content=prompt_buenos_ejemplos)])
        print("Resultado:")
        print(response.content)
        
        # Intentar parsear JSON
        try:
            result_json = json.loads(response.content.strip())
            print("\n✓ JSON válido generado")
            print(f"✓ Campos extraídos: {list(result_json.keys())}")
        except:
            print("\n✗ JSON inválido - formato inconsistente")
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar selección de ejemplos
ejemplos_representativos()

=== SELECCIÓN DE EJEMPLOS REPRESENTATIVOS ===
USANDO EJEMPLOS REPRESENTATIVOS:
Resultado:
```json
{
  "nombre": "María García",
  "empresa": "TechCorp",
  "telefono": "+34 678 901 234",
  "email": "m.garcia@techcorp.es",
  "cargo": "Gerente de Ventas"
}
```

✗ JSON inválido - formato inconsistente


In [5]:
# Técnica 2: Balanceo de Categorías
def balanceo_categorias():
    print("=== BALANCEO DE CATEGORÍAS ===")
    
    # Tarea: Clasificar tickets de soporte
    nuevo_ticket = "Mi aplicación se cierra inesperadamente cuando intento abrir archivos grandes. ¿Pueden ayudarme?"
    
    # Ejemplos balanceados por categoría
    prompt_balanceado = f"""Clasifica cada ticket de soporte en: TÉCNICO, FACTURACIÓN, GENERAL:
    
Ticket: "No puedo acceder a mi cuenta, dice que mi contraseña es incorrecta"
Categoría: TÉCNICO
    
Ticket: "¿Cuándo se procesará mi reembolso del mes pasado?"
Categoría: FACTURACIÓN
    
Ticket: "¿Tienen planes de expandirse a otros países?"
Categoría: GENERAL
    
Ticket: "El botón de exportar datos no funciona en Chrome"
Categoría: TÉCNICO
    
Ticket: "Necesito cambiar el método de pago de mi suscripción"
Categoría: FACTURACIÓN
    
Ticket: "¿Cuál es su política de privacidad de datos?"
Categoría: GENERAL
    
Ticket: "{nuevo_ticket}"
Categoría:"""
    
    print("USANDO EJEMPLOS BALANCEADOS:")
    try:
        response = llm.invoke([HumanMessage(content=prompt_balanceado)])
        resultado = response.content.strip()
        print(f"Clasificación: {resultado}")
        
        # Análisis del balanceo en ejemplos
        ejemplos_tecnico = prompt_balanceado.count("TÉCNICO")
        ejemplos_facturacion = prompt_balanceado.count("FACTURACIÓN")
        ejemplos_general = prompt_balanceado.count("GENERAL")
        
        print(f"\nDistribución de ejemplos:")
        print(f"• TÉCNICO: {ejemplos_tecnico-1} ejemplos")  # -1 porque cuenta el resultado también
        print(f"• FACTURACIÓN: {ejemplos_facturacion-1} ejemplos")
        print(f"• GENERAL: {ejemplos_general-1} ejemplos")
        print(f"✓ Balanceado: 2 ejemplos por categoría")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar balanceo
balanceo_categorias()

=== BALANCEO DE CATEGORÍAS ===
USANDO EJEMPLOS BALANCEADOS:
Clasificación: TÉCNICO

Distribución de ejemplos:
• TÉCNICO: 2 ejemplos
• FACTURACIÓN: 2 ejemplos
• GENERAL: 2 ejemplos
✓ Balanceado: 2 ejemplos por categoría


## Optimización del Número de Ejemplos

¿Cuántos ejemplos necesitamos? Depende de la tarea y complejidad.

In [6]:
# Experimento: 1-shot vs 3-shot vs 5-shot
def optimizar_numero_ejemplos():
    print("=== OPTIMIZACIÓN DEL NÚMERO DE EJEMPLOS ===")
    
    # Tarea: Convertir descripciones casuales a formato técnico
    descripcion = "La página web no carga bien en mi móvil"
    
    # 1-shot
    prompt_1_shot = f"""Convierte descripciones casuales a formato técnico:
    
Casual: "Mi computadora va muy lenta"
Técnico: "Degradación del rendimiento del sistema - posible alto uso de CPU/RAM o fragmentación de disco"
    
Casual: "{descripcion}"
Técnico:"""
    
    # 3-shot
    prompt_3_shot = f"""Convierte descripciones casuales a formato técnico:
    
Casual: "Mi computadora va muy lenta"
Técnico: "Degradación del rendimiento del sistema - posible alto uso de CPU/RAM o fragmentación de disco"
    
Casual: "No puedo enviar emails"
Técnico: "Fallo en servicio SMTP - verificar configuración de servidor de correo saliente"
    
Casual: "La impresora no funciona"
Técnico: "Error de conectividad o driver de dispositivo de impresión - revisar estado de cola y drivers"
    
Casual: "{descripcion}"
Técnico:"""
    
    # 5-shot
    prompt_5_shot = f"""Convierte descripciones casuales a formato técnico:
    
Casual: "Mi computadora va muy lenta"
Técnico: "Degradación del rendimiento del sistema - posible alto uso de CPU/RAM o fragmentación de disco"
    
Casual: "No puedo enviar emails"
Técnico: "Fallo en servicio SMTP - verificar configuración de servidor de correo saliente"
    
Casual: "La impresora no funciona"
Técnico: "Error de conectividad o driver de dispositivo de impresión - revisar estado de cola y drivers"
    
Casual: "El WiFi se desconecta constantemente"
Técnico: "Inestabilidad de conexión inalámbrica - posible interferencia o problema de configuración de red"
    
Casual: "No encuentro mis archivos"
Técnico: "Error de indexación del sistema de archivos - verificar integridad del directorio y permisos"
    
Casual: "{descripcion}"
Técnico:"""
    
    prompts = {
        "1-shot": prompt_1_shot,
        "3-shot": prompt_3_shot,
        "5-shot": prompt_5_shot
    }
    
    for nombre, prompt in prompts.items():
        print(f"\n{nombre.upper()}:")
        print("-" * 20)
        try:
            response = llm.invoke([HumanMessage(content=prompt)])
            resultado = response.content.strip()
            print(f"Resultado: {resultado}")
            
            # Métricas básicas
            tokens_prompt = len(prompt.split())
            tecnicidad = sum(1 for palabra in ['TCP', 'HTTP', 'API', 'DNS', 'CSS', 'JavaScript', 'servidor', 'browser', 'responsive'] if palabra.lower() in resultado.lower())
            
            print(f"• Tokens del prompt: ~{tokens_prompt}")
            print(f"• Nivel técnico: {tecnicidad}/9 términos técnicos")
            
        except Exception as e:
            print(f"Error: {e}")
    
    print("\n=== CONCLUSIONES ===")
    print("• 1-shot: Rápido pero puede ser inconsistente")
    print("• 3-shot: Balance entre costo y calidad")
    print("• 5-shot: Más consistente pero más costoso")
    print("• Regla general: 3-5 ejemplos para la mayoría de tareas")

# Ejecutar optimización
optimizar_numero_ejemplos()

=== OPTIMIZACIÓN DEL NÚMERO DE EJEMPLOS ===

1-SHOT:
--------------------
Resultado: "Problemas de renderización o carga en dispositivos móviles - posible incompatibilidad de diseño responsivo, latencia de red, o errores en el código HTML/CSS/JavaScript"
• Tokens del prompt: ~39
• Nivel técnico: 2/9 términos técnicos

3-SHOT:
--------------------
Resultado: "Tiempos de carga elevados o errores de renderizado en navegador móvil - posible incompatibilidad de diseño responsivo o problemas de conectividad"
• Tokens del prompt: ~79
• Nivel técnico: 0/9 términos técnicos

5-SHOT:
--------------------
Resultado: "Tiempos de carga prolongados o errores de renderizado en navegador móvil - posible incompatibilidad de diseño responsivo o problemas de conectividad móvil"
• Tokens del prompt: ~119
• Nivel técnico: 0/9 términos técnicos

=== CONCLUSIONES ===
• 1-shot: Rápido pero puede ser inconsistente
• 3-shot: Balance entre costo y calidad
• 5-shot: Más consistente pero más costoso
• Regla genera

## Casos de Uso Avanzados

In [8]:
# Caso 1: Generación de Código con Patrones Específicos
def few_shot_codigo():
    print("=== FEW-SHOT PARA GENERACIÓN DE CÓDIGO ===")
    
    # Tarea: Generar funciones Python con docstrings específicos
    nueva_funcion = "función que calcule el área de un círculo"
    
    prompt_codigo = f"""Genera funciones Python con docstrings en formato Google Style:
    
Descripción: función que suma dos números
Código:
```python
def sumar(a: float, b: float) -> float:
    \"\"\"Suma dos números y retorna el resultado.
    
    Args:
        a (float): Primer número a sumar.
        b (float): Segundo número a sumar.
        
    Returns:
        float: La suma de a y b.
        
    Example:
        >>> sumar(3.5, 2.1)
        5.6
    \"\"\"
    return a + b
```
    
Descripción: función que encuentra el máximo en una lista
Código:
```python
def encontrar_maximo(numeros: list[float]) -> float:
    \"\"\"Encuentra el valor máximo en una lista de números.
    
    Args:
        numeros (list[float]): Lista de números a evaluar.
        
    Returns:
        float: El valor máximo de la lista.
        
    Raises:
        ValueError: Si la lista está vacía.
        
    Example:
        >>> encontrar_maximo([1.5, 3.2, 2.1])
        3.2
    \"\"\"
    if not numeros:
        raise ValueError("La lista no puede estar vacía")
    return max(numeros)
```
    
Descripción: {nueva_funcion}
Código:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_codigo)])
        print("FUNCIÓN GENERADA:")
        print(response.content)
        
        # Análisis de calidad
        codigo = response.content
        tiene_type_hints = ': ' in codigo and '->' in codigo
        tiene_docstring = '"""' in codigo
        tiene_args = 'Args:' in codigo
        tiene_returns = 'Returns:' in codigo
        tiene_example = 'Example:' in codigo
        
        print("\n=== ANÁLISIS DE CALIDAD ===")
        print(f"✓ Type hints: {'Sí' if tiene_type_hints else 'No'}")
        print(f"✓ Docstring: {'Sí' if tiene_docstring else 'No'}")
        print(f"✓ Args section: {'Sí' if tiene_args else 'No'}")
        print(f"✓ Returns section: {'Sí' if tiene_returns else 'No'}")
        print(f"✓ Example: {'Sí' if tiene_example else 'No'}")
        
        calidad = sum([tiene_type_hints, tiene_docstring, tiene_args, tiene_returns, tiene_example])
        print(f"\nPuntuación: {calidad}/5 - {'Excelente' if calidad >= 4 else 'Bueno' if calidad >= 3 else 'Necesita mejoras'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar generación de código
few_shot_codigo()

=== FEW-SHOT PARA GENERACIÓN DE CÓDIGO ===
FUNCIÓN GENERADA:
```python
import math

def calcular_area_circulo(radio: float) -> float:
    """Calcula el área de un círculo dado su radio.
    
    Args:
        radio (float): El radio del círculo.
        
    Returns:
        float: El área del círculo.
        
    Raises:
        ValueError: Si el radio es negativo.
        
    Example:
        >>> calcular_area_circulo(3)
        28.274333882308138
    """
    if radio < 0:
        raise ValueError("El radio no puede ser negativo")
    return math.pi * radio ** 2
```

=== ANÁLISIS DE CALIDAD ===
✓ Type hints: Sí
✓ Docstring: Sí
✓ Args section: Sí
✓ Returns section: Sí
✓ Example: Sí

Puntuación: 5/5 - Excelente


In [9]:
# Caso 2: Transformación de Datos Estructurados
def few_shot_transformacion_datos():
    print("=== FEW-SHOT PARA TRANSFORMACIÓN DE DATOS ===")
    
    # Tarea: Convertir datos de empleados a diferentes formatos
    nuevo_empleado = "Carlos Mendez, Desarrollador Backend, 5 años experiencia, especialista en Python y PostgreSQL"
    
    prompt_transformacion = f"""Convierte descripciones de empleados a formato JSON estructurado:
    
Descripción: "Ana López, Diseñadora UX/UI, 3 años experiencia, experta en Figma y Adobe XD"
JSON:
{{
  "nombre": "Ana López",
  "puesto": "Diseñadora UX/UI",
  "experiencia_años": 3,
  "habilidades": ["Figma", "Adobe XD"],
  "departamento": "Diseño",
  "nivel": "Mid"
}}
    
Descripción: "Miguel Torres, QA Engineer Senior, 7 años experiencia, especializado en Selenium y Jest"
JSON:
{{
  "nombre": "Miguel Torres",
  "puesto": "QA Engineer Senior",
  "experiencia_años": 7,
  "habilidades": ["Selenium", "Jest"],
  "departamento": "Calidad",
  "nivel": "Senior"
}}
    
Descripción: "Laura García, Data Scientist, 2 años experiencia, conocimientos en Python y TensorFlow"
JSON:
{{
  "nombre": "Laura García",
  "puesto": "Data Scientist",
  "experiencia_años": 2,
  "habilidades": ["Python", "TensorFlow"],
  "departamento": "Datos",
  "nivel": "Junior"
}}
    
Descripción: "{nuevo_empleado}"
JSON:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_transformacion)])
        print("TRANSFORMACIÓN REALIZADA:")
        print(response.content)
        
        # Validar JSON y estructura
        try:
            data = json.loads(response.content.strip())
            campos_esperados = ["nombre", "puesto", "experiencia_años", "habilidades", "departamento", "nivel"]
            campos_presentes = list(data.keys())
            
            print("\n=== VALIDACIÓN ===")
            print(f"✓ JSON válido: Sí")
            print(f"✓ Campos esperados: {len(campos_esperados)}")
            print(f"✓ Campos presentes: {len(campos_presentes)}")
            print(f"✓ Estructura completa: {'Sí' if set(campos_esperados).issubset(set(campos_presentes)) else 'No'}")
            
            # Validar tipos de datos
            tipo_experiencia = type(data.get('experiencia_años')).__name__
            tipo_habilidades = type(data.get('habilidades')).__name__
            
            print(f"✓ Experiencia es número: {'Sí' if tipo_experiencia == 'int' else 'No'}")
            print(f"✓ Habilidades es lista: {'Sí' if tipo_habilidades == 'list' else 'No'}")
            
        except json.JSONDecodeError:
            print("\n✗ JSON inválido - formato inconsistente")
    
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar transformación
few_shot_transformacion_datos()

=== FEW-SHOT PARA TRANSFORMACIÓN DE DATOS ===
TRANSFORMACIÓN REALIZADA:
```json
{
  "nombre": "Carlos Mendez",
  "puesto": "Desarrollador Backend",
  "experiencia_años": 5,
  "habilidades": ["Python", "PostgreSQL"],
  "departamento": "Desarrollo",
  "nivel": "Mid"
}
```

✗ JSON inválido - formato inconsistente


## Técnicas Avanzadas de Few-Shot

In [10]:
# Técnica 1: Few-Shot con Explicaciones
def few_shot_con_explicaciones():
    print("=== FEW-SHOT CON EXPLICACIONES ===")
    
    # Tarea: Análisis de código SQL
    nuevo_sql = "SELECT COUNT(*) FROM users WHERE age > 25 AND status = 'active'"
    
    prompt_explicado = f"""Analiza queries SQL y explica qué hacen:
    
SQL: "SELECT name, email FROM employees WHERE department = 'IT'"
Explicación: Esta query selecciona los nombres y emails de todos los empleados que trabajan en el departamento de IT. Usa un filtro WHERE para limitar los resultados solo a empleados de IT.
Complejidad: Básica
    
SQL: "SELECT d.name, COUNT(e.id) FROM departments d LEFT JOIN employees e ON d.id = e.dept_id GROUP BY d.name"
Explicación: Esta query cuenta cuántos empleados hay en cada departamento. Usa LEFT JOIN para incluir departamentos sin empleados (mostrarían 0) y GROUP BY para agrupar los resultados por nombre de departamento.
Complejidad: Intermedia
    
SQL: "SELECT * FROM products WHERE price BETWEEN 100 AND 500 ORDER BY price DESC"
Explicación: Esta query obtiene todos los productos con precios entre 100 y 500, ordenados de mayor a menor precio. BETWEEN incluye ambos valores límite (100 y 500).
Complejidad: Básica
    
SQL: "{nuevo_sql}"
Explicación:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_explicado)])
        print("ANÁLISIS GENERADO:")
        print(response.content)
        
        # Verificar si siguió el patrón
        resultado = response.content
        tiene_explicacion = len(resultado) > 50
        menciona_complejidad = 'complejidad' in resultado.lower()
        explica_clausulas = any(palabra in resultado.lower() for palabra in ['where', 'count', 'select'])
        
        print("\n=== ANÁLISIS DEL PATRÓN ===")
        print(f"✓ Explicación detallada: {'Sí' if tiene_explicacion else 'No'}")
        print(f"✓ Menciona complejidad: {'Sí' if menciona_complejidad else 'No'}")
        print(f"✓ Explica cláusulas SQL: {'Sí' if explica_clausulas else 'No'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar few-shot con explicaciones
few_shot_con_explicaciones()

=== FEW-SHOT CON EXPLICACIONES ===
ANÁLISIS GENERADO:
Esta query cuenta cuántos usuarios tienen más de 25 años y están marcados como "activos". Utiliza la función de agregación `COUNT(*)` para contar las filas que cumplen las condiciones especificadas en el filtro `WHERE`, que combina dos criterios: `age > 25` y `status = 'active'`.

**Complejidad:** Básica

=== ANÁLISIS DEL PATRÓN ===
✓ Explicación detallada: Sí
✓ Menciona complejidad: Sí
✓ Explica cláusulas SQL: Sí


In [11]:
# Técnica 2: Few-Shot Progresivo (Aumentando Complejidad)
def few_shot_progresivo():
    print("=== FEW-SHOT PROGRESIVO ===")
    
    # Tarea: Generar expresiones regulares
    nueva_tarea = "expresión regular para validar números de teléfono españoles (+34 XXX XXX XXX)"
    
    prompt_progresivo = f"""Genera expresiones regulares con explicación:
    
Tarea: validar email básico
Regex: ^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{{2,}}$
Explicación: Valida formato básico de email con caracteres permitidos antes y después del @, y dominio con al menos 2 caracteres.
Complejidad: ⭐⭐
    
Tarea: extraer URLs de texto
Regex: https?://(?:[-\w.])+(?::[0-9]+)?(?:/(?:[\w/_.])*(?:\?(?:[\w&=%.])*)?(?:#(?:[\w.])*)?)?)
Explicación: Captura URLs HTTP/HTTPS con dominio, puerto opcional, path, query parameters y fragmentos.
Complejidad: ⭐⭐⭐⭐
    
Tarea: validar fecha formato DD/MM/YYYY
Regex: ^(?:(?:31(\/|-|\.)(?:0?[13578]|1[02]))\1|(?:(?:29|30)(\/|-|\.)(?:0?[13-9]|1[0-2])\2))(?:(?:1[6-9]|[2-9]\d)?\d{{2}})$|^(?:29(\/|-|\.)0?2\3(?:(?:(?:1[6-9]|[2-9]\d)?(?:0[48]|[2468][048]|[13579][26])|(?:(?:16|[2468][048]|[3579][26])00))))$|^(?:0?[1-9]|1\d|2[0-8])(\/|-|\.)(?:(?:0?[1-9])|(?:1[0-2]))\4(?:(?:1[6-9]|[2-9]\d)?\d{{2}})$
Explicación: Valida fechas considerando años bisiestos, días válidos por mes, y diferentes separadores. Muy compleja pero precisa.
Complejidad: ⭐⭐⭐⭐⭐
    
Tarea: {nueva_tarea}
Regex:"""
    
    try:
        response = llm.invoke([HumanMessage(content=prompt_progresivo)])
        print("EXPRESIÓN REGULAR GENERADA:")
        print(response.content)
        
        # Analizar complejidad
        resultado = response.content
        tiene_regex = bool(re.search(r'[\[\](){}.*+?^$|\\]', resultado))
        tiene_explicacion = 'Explicación:' in resultado
        tiene_complejidad = '⭐' in resultado or 'Complejidad:' in resultado
        
        print("\n=== ANÁLISIS ===")
        print(f"✓ Contiene regex: {'Sí' if tiene_regex else 'No'}")
        print(f"✓ Incluye explicación: {'Sí' if tiene_explicacion else 'No'}")
        print(f"✓ Indica complejidad: {'Sí' if tiene_complejidad else 'No'}")
        
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar few-shot progresivo
import re
few_shot_progresivo()

=== FEW-SHOT PROGRESIVO ===
EXPRESIÓN REGULAR GENERADA:
Regex: `^\+34\s\d{3}\s\d{3}\s\d{3}$`

### Explicación:
1. `^`: Indica el inicio de la cadena.
2. `\+34`: Obliga a que el número comience con el prefijo internacional de España (`+34`).
   - El `+` se escapa con una barra invertida (`\`) porque es un carácter especial en regex.
3. `\s`: Representa un espacio obligatorio después del prefijo.
4. `\d{3}`: Acepta exactamente 3 dígitos (números del 0 al 9).
5. `\s`: Representa otro espacio obligatorio.
6. `\d{3}`: Acepta otros 3 dígitos.
7. `\s`: Representa un tercer espacio obligatorio.
8. `\d{3}`: Acepta los últimos 3 dígitos.
9. `$`: Indica el final de la cadena.

### Ejemplos válidos:
- `+34 123 456 789`
- `+34 987 654 321`

### Ejemplos no válidos:
- `+34 123456789` (falta espacios)
- `34 123 456 789` (falta el `+`)
- `+34 123 45678` (número incompleto)

### Complejidad: ⭐⭐
Esta expresión regular es sencilla porque solo valida un formato específico de número de teléfono español.

=

## Mejores Prácticas y Consideraciones

### ✅ Cuándo Usar Few-Shot:
- Formato de salida específico requerido
- Tareas con patrones complejos
- Cuando la consistencia es crítica
- Dominios especializados con ejemplos disponibles

### 🎯 Selección de Ejemplos:
1. **Diversidad**: Cubrir diferentes variaciones de la tarea
2. **Calidad**: Ejemplos perfectos que seguir
3. **Balance**: Representar todas las categorías equitativamente
4. **Claridad**: Ejemplos inequívocos y bien estructurados

### ⚠️ Limitaciones:
- **Costo**: Más tokens = mayor costo
- **Contexto**: Límite de ventana de contexto
- **Sesgos**: Los ejemplos pueden introducir sesgos
- **Overfitting**: Puede ser demasiado rígido

In [12]:
# Ejercicio final: Diseña tu propio few-shot prompt
def ejercicio_few_shot():
    print("=== EJERCICIO: DISEÑA TU FEW-SHOT PROMPT ===")
    print("\nTarea: Crear un sistema que convierta descripciones de bugs en tickets estructurados")
    print("\nRequisitos:")
    print("- Extraer: título, descripción, severidad, componente afectado")
    print("- Formato JSON consistente")
    print("- Clasificar severidad: Alta, Media, Baja")
    print("- Identificar componente: Frontend, Backend, Database, API")
    
    # Ejemplo de bug description para procesar
    bug_ejemplo = """Cuando hago clic en el botón de login después de ingresar credenciales válidas, 
    la página se queda en blanco y no pasa nada. Esto pasa solo en Chrome. El error aparece 
    en la consola como 'TypeError: Cannot read property of undefined'."""
    
    print(f"\nBug a procesar: {bug_ejemplo}")
    print("\nDiseña un few-shot prompt con 3 ejemplos que cubran:")
    print("1. Bug de frontend")
    print("2. Bug de backend/API")
    print("3. Bug de base de datos")
    
    # Template para que el estudiante complete
    template = """
    # TU PROMPT AQUÍ:
    
    Convierte descripciones de bugs a tickets estructurados en JSON:
    
    Bug: "[EJEMPLO 1 - Frontend]"
    Ticket: {
        "titulo": "...",
        "descripcion": "...",
        "severidad": "...",
        "componente": "..."
    }
    
    Bug: "[EJEMPLO 2 - Backend/API]"
    Ticket: {...}
    
    Bug: "[EJEMPLO 3 - Database]"
    Ticket: {...}
    
    Bug: "{bug_description}"
    Ticket:
    """
    
    print(template)
    
    # Prompt de ejemplo bien diseñado
    prompt_ejemplo = f"""Convierte descripciones de bugs a tickets estructurados:
    
Bug: "El botón de buscar no responde cuando hay caracteres especiales en el input. Sale error 'Invalid character' en el navegador."
Ticket: {{
    "titulo": "Botón de búsqueda falla con caracteres especiales",
    "descripcion": "Error de validación en frontend al procesar caracteres especiales en campo de búsqueda",
    "severidad": "Media",
    "componente": "Frontend"
}}
    
Bug: "La API devuelve 500 cuando se hace POST a /users con email duplicado. Debería devolver 400 con mensaje claro."
Ticket: {{
    "titulo": "API retorna código de error incorrecto para email duplicado",
    "descripcion": "Endpoint POST /users devuelve 500 en lugar de 400 para email duplicado",
    "severidad": "Alta",
    "componente": "API"
}}
    
Bug: "Los reportes no cargan desde ayer, timeout en queries que normalmente son rápidas. Posible problema de índices."
Ticket: {{
    "titulo": "Degradación de performance en queries de reportes",
    "descripcion": "Timeout en queries de reportes sugiere problema de optimización o índices faltantes",
    "severidad": "Alta",
    "componente": "Database"
}}
    
Bug: "{bug_ejemplo}"
Ticket:"""
    
    print("\n=== PROMPT DE REFERENCIA ===")
    print(prompt_ejemplo)
    
    # Ejecutar el prompt ejemplo
    print("\n=== RESULTADO DEL PROMPT DE REFERENCIA ===")
    try:
        response = llm.invoke([HumanMessage(content=prompt_ejemplo)])
        print(response.content)
    except Exception as e:
        print(f"Error: {e}")

# Ejecutar ejercicio
ejercicio_few_shot()

=== EJERCICIO: DISEÑA TU FEW-SHOT PROMPT ===

Tarea: Crear un sistema que convierta descripciones de bugs en tickets estructurados

Requisitos:
- Extraer: título, descripción, severidad, componente afectado
- Formato JSON consistente
- Clasificar severidad: Alta, Media, Baja
- Identificar componente: Frontend, Backend, Database, API

Bug a procesar: Cuando hago clic en el botón de login después de ingresar credenciales válidas, 
    la página se queda en blanco y no pasa nada. Esto pasa solo en Chrome. El error aparece 
    en la consola como 'TypeError: Cannot read property of undefined'.

Diseña un few-shot prompt con 3 ejemplos que cubran:
1. Bug de frontend
2. Bug de backend/API
3. Bug de base de datos

    # TU PROMPT AQUÍ:
    
    Convierte descripciones de bugs a tickets estructurados en JSON:
    
    Bug: "[EJEMPLO 1 - Frontend]"
    Ticket: {
        "titulo": "...",
        "descripcion": "...",
        "severidad": "...",
        "componente": "..."
    }
    
    Bug: "[E

## Conceptos Clave Aprendidos

1. **Few-shot prompting** mejora consistencia y control de formato
2. **Selección de ejemplos** es crucial para el éxito
3. **Balance de categorías** evita sesgos en clasificación
4. **3-5 ejemplos** es generalmente óptimo para la mayoría de tareas
5. **Costo vs. calidad** debe considerarse en la implementación

## Próximos Pasos

En el siguiente notebook exploraremos **Chain-of-Thought Prompting**, una técnica que hace que el modelo "piense en voz alta" y muestre su razonamiento paso a paso, especialmente útil para problemas complejos y matemáticos.

### Para Practicar:
1. Experimenta con diferentes números de ejemplos
2. Prueba diferentes formas de estructurar los ejemplos
3. Compara resultados con y sin few-shot
4. Mide la consistencia en múltiples ejecuciones